# Operação de All Call Lists - CMDB: Retorno de um Terceiro Arquivo

In [1]:
import subprocess
import sys

# Instalação de libs caso não estejam na máquina do desenvolvedor
try:
    import pandas as pd
except ImportError:
    %pip install pandas
finally:
    import pandas as pd

try:
    from pyfiglet import Figlet
except ImportError:
    %pip install pyfiglet
finally:
    from pyfiglet import Figlet

In [ ]:
f = Figlet(width=100)
print(f.renderText("Script made by\nVitor R. G. Gomes - Kyndryl\n2022\n"))

In [ ]:
# Import de arquivos necessários para rodar o script
AllJobsPath = 'ALLJOBS.txt'
CMDBJobsPath = 'CMDB.txt'
hfdCallListPath = 'hfd.xlsx'
hliCallListPath = 'hli.xlsx'
devCallListPath = 'dev.xlsx'
bldCallListPath = 'bld.xlsx'

In [127]:
print('Abrindo arquivos...')
with open('AllTeste.txt', 'r') as f1:
  allCallLists = f1.read().split('\n')

with open('cmdbTeste.txt', 'r') as f1:
  cmdb = f1.read().split('\n')

In [128]:
print("Verificando quais arquivos já estão no CMDB...")
AllCallListsMinusCMDB = [line for line in allCallLists if line not in cmdb]
print("Terceiro arquivo:", len(AllCallListsMinusCMDB))
f = open("AllCallListsMinusCMDB.txt", "a")
f.truncate(0)
f.write("Jobs que estão na Call List, mas não estão no CMDB ainda:\n")
for jobName in AllCallListsMinusCMDB:
    f.write(jobName+"\n")
f.close()
AllCallListsMinusCMDB

['a', 'b', 'c', 'd', 'e', 'g']

# Comparação com as call lists e retorno de um quarto arquivo com o Job Name junto com instructions ou Job name e NOT Found se Instructions não forem achadas

In [130]:
print("Abrindo Call Lists...")
hfdCallListDf = pd.read_excel(hfdCallListPath)
hliCallListDf = pd.read_excel(hliCallListPath)
devCallListDf = pd.read_excel(devCallListPath)
bldCallListDf = pd.read_excel(bldCallListPath)
print("hfd:", len(hfdCallListPath))
print("hli:", len(hliCallListPath))
print("dev:", len(devCallListPath))
print("bld:", len(bldCallListPath))

In [131]:
hfdCallListDf.head()

,Job,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,a,t,e,f,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b,1,2,3,4,5.0,6.0,7.0,8.0,9.0,10.0,11.0
2,c,1,2,3,4,5.0,NaN,NaN,NaN,NaN,NaN,NaN
3,d,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
dfFinalResult = pd.DataFrame(columns=['Job Name', 'Instructions'])
dfFinalResult

,Job Name,Instructions


In [150]:
print("Verificando quais Jobs estão nas Call Lists e pegando suas instruções...")
for idxJob, job in enumerate(AllCallListsMinusCMDB):
    print("Cont:", idxJob)
    achou = False
    detail = ''
    if job != '':
        if not achou:
            for i, j in enumerate(hfdCallListDf['Job']):
                if(j == job):
                    jobName = job
                    achou = True
                    for idx, val in enumerate(hfdCallListDf.iloc[i, :]):
                        if(idx > 0 and not pd.isna(val)):
                            detail += str(idx)+"- "+str(val)+'\n'
                    if not detail:
                        detail = 'Instructions not found'
                    break
        if not achou:
            for i, j in enumerate(hliCallListDf['Job']):
                if(j == job):
                    jobName = job
                    achou = True
                    for idx, val in enumerate(hliCallListDf.iloc[i, :]):
                        if(idx > 0 and not pd.isna(val)):
                            detail += str(idx)+"- "+str(val)+'\n'
                    if not detail:
                        detail = 'Instructions not found'
                    break
        if not achou:
            for i, j in enumerate(devCallListDf['Job']):
                if(j == job):
                    jobName = job
                    achou = True
                    for idx, val in enumerate(devCallListDf.iloc[i, :]):
                        if(idx > 0 and not pd.isna(val)):
                            detail += str(idx)+"- "+str(val)+'\n'
                    if not detail:
                        detail = 'Instructions not found'
                    break
        if not achou:
            for i, j in enumerate(bldCallListDf['Job']):
                if(j == job):
                    jobName = job
                    achou = True
                    for idx, val in enumerate(bldCallListDf.iloc[i, :]):
                        if(idx > 0 and not pd.isna(val)):
                            detail += str(idx)+"- "+str(val)+'\n'
                    if not detail:
                        detail = 'Instructions not found'
                    break
        if not achou:
            jobName = job
            detail = "Job not found in any call list"

        dfFinalResult.loc[dfFinalResult.shape[0]] = [jobName, detail]

dfFinalResult.to_excel("output.xlsx",
                       sheet_name='Output')

In [151]:
dfFinalResult.replace(r'\n', ' ', regex=True, inplace=True)
dfFinalResult

,Job Name,Instructions
0,a,1- t 2- e 3- f 4- g
1,b,1- 1 2- 2 3- 3 4- 4 5- 5.0 6- 6.0 7- 7.0 8- 8....
2,c,1- 1 2- 2 3- 3 4- 4 5- 5.0
3,d,1- 1
4,e,Instructions not found
5,g,Job not found in any call list


In [153]:
print("Escrevendo arquivo final...")
with open('RelacaoFinalCMDBCallList.txt', 'a', encoding="utf-8") as f:
    f.truncate(0)
    dfAsString = dfFinalResult.to_string(header=True, index=True)
    f.write(dfAsString)
print("Tudo pronto, finalizando programa...\nSee ya :D")
dfFinalResult.head()

,Job Name,Instructions
0,a,1- t 2- e 3- f 4- g
1,b,1- 1 2- 2 3- 3 4- 4 5- 5.0 6- 6.0 7- 7.0 8- 8....
2,c,1- 1 2- 2 3- 3 4- 4 5- 5.0
3,d,1- 1
4,e,Instructions not found
